In [ ]:
from google.colab import drive
# Mount google drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP/2
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Second Competition/final
Anything_goes.ipynb  justtest.ipynb  test.tsv  train.tsv  untitled_project/


In [ ]:
import pandas as pd
# Load dataset file
df_total = pd.read_csv("train.tsv", sep='\t', names=["word", "tag"])
df_total.head()

,word,tag
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N


In [ ]:
import csv

df_test = pd.read_csv("test.tsv", sep='\t', names=["word", "tag"], quoting=csv.QUOTE_NONE)
final_test = [[(df_test.loc[i,'word'], df_test.loc[i,'tag'])] for i in range(len(df_test))]

In [ ]:
# df2 = df_total[:15000]
df2 = df_total.copy()
df2.describe()
print((df2['tag'].value_counts()))
lbl = df2['tag'].value_counts().max()
max_label = df2['tag'][lbl]
max_label

N    4339849
S     493102
U     165818
H      40569
T      17721
Name: tag, dtype: int64


'N'

In [ ]:
# converting input shape for tagger
df3 = [[(df2.loc[i,'word'], df2.loc[i,'tag'])] for i in range(len(df2))]

In [ ]:
# creating train and test datasets - source: https://colab.research.google.com/drive/1pAuj6j5UAzBrGHPmgP5IufT35O9E6lvt?usp=sharing
split = int(len(df3)*0.9)
train = df3[:split]
test = df3[split:]
len(train)

4551353

**Unigram tagger**

In [ ]:
# unigram tagger 
import nltk
unigram_tagger = nltk.UnigramTagger(train)
print("Unigram tagger accuracy without backoff tagger is: ", unigram_tagger.accuracy(test))

Unigram tagger accuracy without backoff tagger is:  0.8941875318861157


In [ ]:
from nltk.tag import SequentialBackoffTagger
from nltk.tag import DefaultTagger 
from nltk.tag import UnigramTagger 

back_tagger = DefaultTagger(max_label)
unigram_tagger2 = UnigramTagger(train, backoff = back_tagger)
print("Unigram tagger accuracy with backoff tagger is: ", unigram_tagger2.accuracy(test))

Unigram tagger accuracy with backoff tagger is:  0.907707244920962


In [ ]:
# accuracy on final test set
print("Unigram tagger accuracy with backoff tagger is: ", unigram_tagger2.accuracy(final_test))

Unigram tagger accuracy with backoff tagger is:  0.908256


**RNN**

In [ ]:
df_rnn = df_total[:15000]
df_rnn2 = [[(df_rnn.loc[i,'word'], df_rnn.loc[i,'tag'])] for i in range(len(df_rnn))]

# creating vocab and tags lists from df for word2index and tag2index - source: https://colab.research.google.com/drive/1D8aeXQVb_aVcfr7thA7UYlmD0Tq_XjPU?usp=sharing
vocab = list(df_rnn['word'])
vocab.append('<PAD>')

tags = list(df_rnn['tag'])
tags.append('<PAD>')

print(len(vocab))
print(len(tags))

15001
15001


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# source: https://colab.research.google.com/drive/1D8aeXQVb_aVcfr7thA7UYlmD0Tq_XjPU?usp=sharing
max_len = 5
word2index = {w: i for i, w in enumerate(vocab)}
tag2index = {t: i for i, t in enumerate(tags)}
onehot = [[word2index[w[0]] for w in s] for s in df_rnn2]
X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(vocab)-1)

In [ ]:
print(len(onehot))
print(type(onehot))
print(X.shape)

15000
<class 'list'>
(15000, 1)


In [ ]:
print(len(X))
print(type(X))
print(X.shape)
X[1]

15000
<class 'numpy.ndarray'>
(15000, 1)


array([14746], dtype=int32)

In [ ]:
from tensorflow.keras.utils import to_categorical

onehot_y = [[tag2index[w[1]] for w in s] for s in df_rnn2]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
y = to_categorical(y, num_classes=len(tags))

In [ ]:
print(len(onehot_y))
print(type(onehot_y))
onehot_y[1]

15000
<class 'list'>


[14999]

In [ ]:
print(len(y))
print(type(y))
print(y.shape)
y[1]

15000
<class 'numpy.ndarray'>
(15000, 15001)


array([0., 0., 0., ..., 0., 1., 0.], dtype=float32)

In [ ]:
# creaint train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

13500
1500
13500
1500
(13500, 1)
(1500, 1)
(13500, 15001)


In [ ]:
%pip install -q -U keras-tuner
import keras_tuner as kt

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 

# source: Tensorflow tutorials https://www.tensorflow.org/tutorials/keras/keras_tuner

def model_builder(hp):
  model = keras.Sequential()

  # Tune the number of units in the first Dense layer - Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Embedding(input_dim=len(vocab), output_dim=50, input_length=max_len))
  model.add(Bidirectional(LSTM(units=hp_units, return_sequences=True, recurrent_dropout=0.1)))
  model.add(TimeDistributed(Dense(len(tags), activation="softmax")))


  # Tune the learning rate for the optimizer - Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) 
  # model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=3,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_train, y_train, epochs= 3, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print("The optimal number of units in the first densely-connected layer is: ", best_hps.get('units') , " and the optimal learning rate for the optimizer is "
, best_hps.get('learning_rate'))

The optimal number of units in the first densely-connected layer is:  32  and the optimal learning rate for the optimizer is  0.0001


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for number of epochs
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
tf.keras.utils.plot_model

<function keras.utils.vis_utils.plot_model(model, to_file='model.png', show_shapes=False, show_dtype=False, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96, layer_range=None, show_layer_activations=False)>

In [ ]:
# x_train.reshape
history = model.fit(X_train, y_train, epochs= 5, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

**HMM**

In [ ]:
# Import the toolkit and tags
import nltk

# Import HMM module
from nltk.tag import hmm

# Setup a trainer with default(None) values and train with the data
tagger = nltk.HiddenMarkovModelTagger.train(train)
# Prints the basic data about the tagger
print(tagger)
# accuracy on test subset of training data
print(tagger.accuracy(test))

<HiddenMarkovModelTagger 5 states and 118706 output symbols>
0.9078239134991477


In [ ]:
print(tagger.accuracy(final_test))

0.908514
